### Setting Up Azure Machine Learning Environment for Microorganism Image Classification

This script configures the Azure Machine Learning environment and sets up a pipeline for the classification of microorganism images

In [1]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

### Connecting to Azure Machine Learning Workspace

In [2]:
# Load Azure ML workspace
ws = Workspace.from_config()

# Azure ML workspace
print(f"Connected to Azure ML workspace: {ws.name} in region {ws.location}")

Connected to Azure ML workspace: azuremlworkspace in region eastus2


### Environment Configuration

Configure the computational environment, specifying dependencies and settings required for the image classification workflow.

In [3]:
# Create a run configuration
run_config = RunConfiguration()

# Set up the Conda environment
conda_dependencies = CondaDependencies.create(pip_packages=['azureml-defaults', 'scikit-learn', 
                                                    'tensorflow', 'pandas', 'numpy', 'pillow'])
run_config.environment.python.conda_dependencies = conda_dependencies

### Defining Azure Machine Learning Compute

Define the computing resources necessary for training and inference tasks, optimizing scalability and performance.

In [4]:
# Define your compute target
compute_target_name = 'AzureMLpipelineCompute'

# Check if the compute target already exists in the workspace
if compute_target_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_target_name]
    print(f"Using existing compute target: {compute_target_name}")
else:
    # If the compute target doesn't exist, provision a new one
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
    compute_target = ComputeTarget.create(ws, compute_target_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
    print(f"New compute target provisioned: {compute_target_name}")

Using existing compute target: AzureMLpipelineCompute


### Creating an Experiment

In [5]:
# Define your experiment
experiment_name = 'sample_prediction'
experiment = Experiment(workspace=ws, name=experiment_name)

### Register the Trained Model

In [6]:
from azureml.core import Model

# Register the trained model
model = Model.register(workspace=ws, model_name='trained_model.h5', model_path='trained_model.h5')


Registering model trained_model.h5


In [7]:
registered_model = Model(ws, name='trained_model.h5', version=1)
print(registered_model.name)

trained_model.h5


### Defining Inference Script in Azure ML Pipeline

In [8]:
# Define the PythonScriptStep
script_step = PythonScriptStep(
    script_name='inference_script.py',
    arguments=['--model_name', registered_model.name, '--label_encoder_name', 'label_encoder.pkl', 
                '--image_path', 'Amoeba_0_28.jpg'],
    compute_target='AzureMLpipelineCompute',  # Specify compute target
    source_directory='.',  # the script is in the same directory as this code
    runconfig=run_config
)

### Creating the Pipeline

In [9]:
# Create the pipeline
pipeline_steps = [script_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)

###  Executing the Pipeline

In [10]:
# Run the experiment with the PythonScriptStep
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

StepRun( inference_imageClassPrediction.py ) Status: Running
